In [ ]:
import os
import dotenv

dotenv.load_dotenv()

MAIN_SAFE = "0x4c190e3C582e877942007Db3009feACc1Da6C481"
METAMASK_EOA = os.getenv("WALLET_ADDRESS")
POLYMARKET_SAFE = "0x2617746e3Da3cC26EeD139d873f0A05137023e40"
TRADER_EOA_PRIVATE_KEY = os.getenv("TRADER_EOA_PRIVATE_KEY")

ACC_2_ADDRESS = "0xfd4fac895d20912b5d3abdd8695d891baa5e90d5"
ACC_2_PRIVATE_KEY = os.getenv("ACC_2_PRIVATE_KEY")
USDCE_ADDRESS = "0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174"
PRIVATE_KEY = os.getenv("PRIVATE_KEY")

POLYGON_RPC = os.getenv("POLYGON_RPC")  # Polygon rpc url

In [ ]:
from eth_abi import encode as abi_encode
from eth_utils import keccak, to_bytes

# --------------------------------------------------
# Input values
# --------------------------------------------------
CREATE_PROXY_TYPEHASH = (
    "0xdee5f5588156b735c3bff14a54c9acefc845807cec91b7fd0809fa3deccab363"
)
domainSeparator = "0x90ceade07f5d7357e2318a68cbe63334e9b333b455678f14083847d611c9c54b"

paymentToken = "0x0000000000000000000000000000000000000000"
payment = 0
paymentReceiver = "0x0000000000000000000000000000000000000000"

# Convert to raw bytes
typehash_bytes = to_bytes(hexstr=CREATE_PROXY_TYPEHASH)
domain_bytes = to_bytes(hexstr=domainSeparator)


# --------------------------------------------------
# 1. structHash = keccak256(abi.encode(...))
# --------------------------------------------------
encoded_struct = abi_encode(
    ["bytes32", "address", "uint256", "address"],
    [
        typehash_bytes,  # MUST be raw bytes, not hex string
        paymentToken,
        payment,
        paymentReceiver,
    ],
)

structHash = keccak(encoded_struct)
print("structHash =", structHash.hex())

# --------------------------------------------------
# 2. digest = keccak256(abi.encodePacked(...))
# --------------------------------------------------
digest = keccak(b"\x19\x01" + domain_bytes + structHash)

print("digest     =", digest.hex())

In [ ]:
from eth_account import Account
from eth_utils import to_bytes, encode_hex
from eth_utils import to_hex

# ---------- INPUTS (CHANGE THESE) ----------
digest = (
    digest  # your final digest = keccak256("\x19\x01" || domainSeparator || structHash)
)
private_key = TRADER_EOA_PRIVATE_KEY

# Ensure private key is valid
acct = Account.from_key(private_key)

# Convert digest hex → bytes
# digest_bytes = to_bytes(hexstr=digest)
digest_bytes = digest

# Sign the digest directly
signature = Account._sign_hash(digest_bytes, private_key=private_key)


v = signature.v


# Convert r and s to 32-byte hex strings
def to_bytes32_hex(value_int):
    return "0x" + value_int.to_bytes(32, byteorder="big").hex()


r = to_bytes32_hex(signature.r)
s = to_bytes32_hex(signature.s)

# Combined signature (optional)
signature_hex = signature.signature.hex()

print("Signer:", acct.address)
print("v:", v)
print("r:", r)
print("s:", s)
print("signature:", signature_hex)